In [6]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag

In [41]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [42]:
copy = gg1.copy(deep=True)
copy.head(5)

,text,timestamp_ms,user,id
0,just had to scramble to find a golden globes s...,2015-01-11 22:20:13.011,"{'id': 19904553, 'screen_name': 'baumbaTz'}",554402424728072192
1,RT @ENews: Show us how you're watching the #Go...,2015-01-11 22:25:13.824,"{'id': 85020476, 'screen_name': 'devinaaaprilia'}",554403686428864513
2,@danaKStew @50ShadesWorldcm @ScarletteDrake Al...,2015-01-11 22:25:13.869,"{'id': 1433214326, 'screen_name': 'jamiedornan...",554403686617985024
3,RT @lisarinna: When your husband tells you tha...,2015-01-11 22:25:13.928,"{'id': 505974820, 'screen_name': 'WyAttDoRiAn'}",554403686865440769
4,“@goldenglobes: Creating multiple mini Moët Mo...,2015-01-11 22:25:14.067,"{'id': 146599313, 'screen_name': 'Rascal207'}",554403687448469504


In [43]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\"]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))
copy.head()

,text,timestamp_ms,user,id
0,just had to scramble to find a golden globes s...,2015-01-11 22:20:13.011,"{'id': 19904553, 'screen_name': 'baumbaTz'}",554402424728072192
1,RT : Show us how youre watching the -- tweet ...,2015-01-11 22:25:13.824,"{'id': 85020476, 'screen_name': 'devinaaaprilia'}",554403686428864513
2,Also Red Carpet um 12 &amp; die Show vill um 1,2015-01-11 22:25:13.869,"{'id': 1433214326, 'screen_name': 'jamiedornan...",554403686617985024
3,RT : When your husband tells you that you Are ...,2015-01-11 22:25:13.928,"{'id': 505974820, 'screen_name': 'WyAttDoRiAn'}",554403686865440769
4,“: Creating multiple mini Moët Moments on the ...,2015-01-11 22:25:14.067,"{'id': 146599313, 'screen_name': 'Rascal207'}",554403687448469504


In [44]:
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])
df = df.loc[df['text'].str.contains('host')]
print(len(df))
df.head()

1754153
753123
8285


,text
35,Can Amy and Tina just host everything from now...
49,I dont like the hosts
150,Where did Amy and Tina go Uhhh arent they hosting
334,RT : I hope kendall goes to the golden globes ...
346,Amy and Tina should host all the award shows


In [45]:
def findProperNouns(sen):
    tagged_sent = pos_tag(sen.split())
    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    return propernouns
df['ProperNouns'] = df['text'].apply(lambda x: findProperNouns(x))
df.head(10)

,text,ProperNouns
35,Can Amy and Tina just host everything from now...,"[Amy, Tina]"
49,I dont like the hosts,[]
150,Where did Amy and Tina go Uhhh arent they hosting,"[Amy, Tina, Uhhh]"
334,RT : I hope kendall goes to the golden globes ...,[]
346,Amy and Tina should host all the award shows,"[Amy, Tina]"
561,Show over I host next year Lol,[Lol]
565,next years hosts: and,[]
614,We love and hosting look a little better tha...,[]
662,Is this the last time Amy and Tina are hosting...,"[Amy, Tina]"
821,Me when people start suggesting Kristen Wiig a...,"[Kristen, Wiig, Bill, Hader]"


In [46]:
ted = df['ProperNouns'].iloc[0]
for i in range(0, len(ted), 2):
    name = ted[i] + ' ' + ted[i+1]
    print(name)

Amy Tina


In [47]:
NGRAMS_DICT = {}
for i in range(len(df)):
    PN = df['ProperNouns'].iloc[i]
    if len(PN) % 2 == 0: 
        for j in range(0, len(PN), 2):
            name = PN[j] + ' ' + PN[j+1]
            if name in NGRAMS_DICT:
                NGRAMS_DICT[name] += 1
            else:
                NGRAMS_DICT[name] = 1

In [48]:
cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
cd

[('Golden Globes', 809),
 ('Tina Fey', 656),
 ('Amy Poehler', 603),
 ('Bill Hader', 465),
 ('Tina Amy', 430),
 ('Kristen Wiig', 428),
 ('Amy Tina', 156),
 ('Ricky Gervais', 129),
 ('Theyre December', 71),
 ('Fey Amy', 67),
 ('Golden Globe', 54),
 ('&amp; Amy', 50),
 ('Kristin Wiig', 50),
 ('Kevin Hart', 45),
 ('Lily Tomlin', 45),
 ('Jane Fonda', 42),
 ('Bill Cosby', 39),
 ('Wiig Hader', 36),
 ('Fey &amp;', 33),
 ('Fey Poehler', 29),
 ('Margaret Cho', 29),
 ('Tina &amp;', 29),
 ('Kristen Bill', 28),
 ('Golden Globes:', 26),
 ('Amy Poehlers', 25),
 ('&amp; Tina', 24),
 ('George Clooney', 22),
 ('Tina Amys', 22),
 ('North Korea', 19),
 ('Melissa McCarthy', 19),
 ('Chris Pratt', 17),
 ('Amy &amp;', 17),
 ('RT PHENOMENAL', 17),
 ('Red Carpet', 17),
 ('Kristen Wigg', 14),
 ('Wiig Bill', 14),
 ('Globe Awards', 14),
 ('Watch Tina', 13),
 ('Meryl Streep', 13),
 ('Anna Faris', 12),
 ('Bill Kristen', 12),
 ('Im Tina', 12),
 ('Sony Cosby', 12),
 ('Poehler Golden', 11),
 ('Amy Tinas', 10),
 ('Kinde

In [50]:
top3 = [i[0] for i in cd[0:3]]
top3

['Golden Globes', 'Tina Fey', 'Amy Poehler']